In [1]:
import pandas as pd
import datetime

In [26]:
bus = pd.read_csv('../data_original/headway_data.csv', 
                  dtype = {}, 
                  parse_dates = ['DATE']
                  )

In [27]:
bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350328 entries, 0 to 350327
Data columns (total 30 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   CALENDAR_ID                350328 non-null  int64         
 1   SERVICE_ABBR               350328 non-null  int64         
 2   ADHERENCE_ID               350328 non-null  int64         
 3   DATE                       350328 non-null  datetime64[ns]
 4   ROUTE_ABBR                 350328 non-null  int64         
 5   BLOCK_ABBR                 350328 non-null  int64         
 6   OPERATOR                   350328 non-null  int64         
 7   TRIP_ID                    350328 non-null  int64         
 8   OVERLOAD_ID                350328 non-null  int64         
 9   ROUTE_DIRECTION_NAME       350328 non-null  object        
 10  TIME_POINT_ABBR            350328 non-null  object        
 11  ROUTE_STOP_SEQUENCE        350317 non-null  float64 

In [28]:
bus.head()

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,ACTUAL_HDWY,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS
0,120230801,1,99457890,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,2,6.500000
1,120230801,1,99457891,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,9,0.000000
2,120230801,1,99457892,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,19,0.000000
3,120230801,1,99457893,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,NaN,NaN,0,0,1,0,NaN,0,35,0.000000
4,120230801,1,99457894,2023-08-01,22,2200,1040,345105,0,FROM DOWNTOWN,...,NaN,NaN,0,0,1,0,0.0,0,36,12.866666


In [ ]:
# subset bus to just TRIP_EDGE == 1 or 2
# edge_1 = bus[bus['TRIP_EDGE'] == 1]
# edge_2 = bus[bus['TRIP_EDGE'] == 2]
# print(edge_1['TRIP_EDGE'].value_counts())
# print(edge_2['TRIP_EDGE'].value_counts())
# print(edge_1['TRIP_ID'].nunique())
# print(edge_2['TRIP_ID'].nunique())

In [29]:
bus['OVERLOAD_ID'].value_counts()

OVERLOAD_ID
0    348424
1      1559
2       278
3        60
4         7
Name: count, dtype: int64

In [30]:
# add column with concat trip id and date to create actual unique trip id
bus['trip_date'] = bus['TRIP_ID'].astype('str') + bus['CALENDAR_ID'].astype('str')

bus.head(1)

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,trip_date
0,120230801,1,99457890,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,NaN,0,0,1,0,0.0,0,2,6.5,345104120230801


In [46]:
bus['TRIP_EDGE'].value_counts()

TRIP_EDGE
0    215185
2     67728
1     67415
Name: count, dtype: int64

In [31]:
# take out trip which are non-scheduled services
no_overload = bus[bus['OVERLOAD_ID'] == 0]
no_overload.shape

(348424, 31)

In [32]:
# after taking out trips where overload_id != 0 (0 is for scheduled service, anything else means it was added often to cover a broken down bus)
# subset bus to just TRIP_EDGE == 1 or 2
edge_1 = no_overload[no_overload['TRIP_EDGE'] == 1]
edge_2 = no_overload[no_overload['TRIP_EDGE'] == 2]
print(edge_1['TRIP_EDGE'].value_counts())
print(edge_2['TRIP_EDGE'].value_counts())
print(edge_1['trip_date'].value_counts())
print(edge_2['trip_date'].nunique())

TRIP_EDGE
1    67355
Name: count, dtype: int64
TRIP_EDGE
2    67354
Name: count, dtype: int64
trip_date
345104120230801    1
350271120230911    1
350273120230911    1
350274120230911    1
350275120230911    1
                  ..
351238120230821    1
351239120230821    1
351240120230821    1
351241120230821    1
353449120230930    1
Name: count, Length: 67355, dtype: int64
67354


In [33]:
# unique to each row but not for a trip as a whole
bus['ADHERENCE_ID'].value_counts()

ADHERENCE_ID
99457890     1
100624112    1
100624120    1
100624119    1
100624118    1
            ..
100035186    1
100035185    1
100035184    1
100035183    1
100406614    1
Name: count, Length: 350328, dtype: int64

In [34]:
# subset to only include trips with exactly one start (1) and one end (2)

# group by trip and count occurence of each value present in TRIP_EDGE
trip_ids = bus[bus['TRIP_EDGE'].isin([1, 2])]
print(trip_ids['TRIP_EDGE'].nunique())

# selects trips with exactly one edge == 1 and one edge == 2
counts = trip_ids.groupby(['trip_date', 'TRIP_EDGE']).size().unstack(fill_value = 0)
valid_trips = counts[(counts[1] == 1) & (counts[2] == 1)].index
print(valid_trips.shape)

# subset
complete_trips = bus[bus['trip_date'].isin(valid_trips)]

2
(67001,)


In [51]:
# subset to only include trips with exactly one start (1) and one end (2)
# using no_overload df

# group by trip and count occurence of each value present in TRIP_EDGE
trip_ids_no = no_overload[no_overload['TRIP_EDGE'].isin([1, 2])]
print(trip_ids_no['TRIP_EDGE'].nunique())

# selects trips with exactly one edge == 1 and one edge == 2
counts_no = trip_ids_no.groupby(['trip_date', 'TRIP_EDGE']).size().unstack(fill_value = 0)
valid_trips_no = counts_no[(counts_no[1] == 1) & (counts_no[2] == 1)].index
print(valid_trips_no.shape)

# subset
complete_trips_no = no_overload[no_overload['trip_date'].isin(valid_trips_no)]

2
(67351,)


In [52]:
trip_ids_no['trip_date'].value_counts()

trip_date
345104120230801    2
350269120230911    2
350271120230911    2
350272120230911    2
350273120230911    2
                  ..
353354120230815    1
352124120230804    1
353048120230929    1
353049120230929    1
352011120230919    1
Name: count, Length: 67358, dtype: int64

In [50]:
trip_ids['trip_date'].value_counts()

trip_date
350169120230909    4
352285120230929    4
350277120230913    4
352790120230804    4
352999120230915    4
                  ..
351344120230830    1
352714120230810    1
352729120230901    1
352126120230804    1
352010120230919    1
Name: count, Length: 67370, dtype: int64

In [35]:
complete_trips.head()

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,HDWY_DEV,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,trip_date
0,120230801,1,99457890,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,NaN,0,0,1,0,0.0,0,2,6.500000,345104120230801
1,120230801,1,99457891,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,NaN,0,0,1,0,0.0,0,9,0.000000,345104120230801
2,120230801,1,99457892,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,NaN,0,0,1,0,0.0,0,19,0.000000,345104120230801
3,120230801,1,99457893,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,NaN,0,0,1,0,NaN,0,35,0.000000,345104120230801
4,120230801,1,99457894,2023-08-01,22,2200,1040,345105,0,FROM DOWNTOWN,...,NaN,0,0,1,0,0.0,0,36,12.866666,345105120230801


In [53]:
complete_trips_no['TRIP_EDGE'].value_counts()

TRIP_EDGE
0    213711
1     67351
2     67351
Name: count, dtype: int64

In [37]:
type(complete_trips['SCHEDULED_TIME'][0])

str

In [40]:
# add weekday column (derived from date and useful for identifying days where the service is other than expected, probably just Labor Day)
# also adding day name
complete_trips['weekday'] = complete_trips['DATE'].dt.dayofweek
complete_trips['day_name'] = complete_trips['DATE'].dt.day_name()

/var/folders/xm/vmtk5pms0jzb2kmtm00lwtn80000gn/T/ipykernel_12508/4152282636.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_trips['weekday'] = complete_trips['DATE'].dt.dayofweek
/var/folders/xm/vmtk5pms0jzb2kmtm00lwtn80000gn/T/ipykernel_12508/4152282636.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_trips['day_name'] = complete_trips['DATE'].dt.day_name()


In [41]:
complete_trips.head()

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT,STOP_CANCELLED,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,trip_date,weekday,day_name
0,120230801,1,99457890,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,0,1,0,0.0,0,2,6.500000,345104120230801,1,Tuesday
1,120230801,1,99457891,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,0,1,0,0.0,0,9,0.000000,345104120230801,1,Tuesday
2,120230801,1,99457892,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,0,1,0,0.0,0,19,0.000000,345104120230801,1,Tuesday
3,120230801,1,99457893,2023-08-01,22,2200,1040,345104,0,TO DOWNTOWN,...,0,1,0,NaN,0,35,0.000000,345104120230801,1,Tuesday
4,120230801,1,99457894,2023-08-01,22,2200,1040,345105,0,FROM DOWNTOWN,...,0,1,0,0.0,0,36,12.866666,345105120230801,1,Tuesday


In [43]:
complete_trips.groupby('trip_date').nunique()

67001

In [44]:
complete_trips['TRIP_ID'].nunique()

3987

In [58]:
complete_trips_no['TRIP_ID'].nunique()

3987

complete_trips_no are complete trips (with both a 1 and a 2) only including scheduled service (overhead id == 0)
This results in ~ 60 fewer complete trips than including other overhead ids, however there will be duplicate trip_date ids since the replacement service buses are given the same trip id as the original scheduled bus

In [59]:
complete_trips_no.to_csv('../data/complete_scheduled_trips', index = False)